# Implementation

In [ ]:
import numpy as np
import dolfinx
import meshio

gmshFlag = 1

fileName = "Incrompressible-Schrodinger-Flow/mesh/BFS.msh"

if gmshFlag == 1:
  msh = meshio.read(fileName)

  def create_mesh(mesh, cell_type, prune_z=False):
      cells = mesh.get_cells_type(cell_type)
      # cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
      points = mesh.points[:,:2] if prune_z else mesh.points
      out_mesh = meshio.Mesh(points=points, cells={cell_type: cells})
      #, cell_data={"name_to_read":[cell_data]})
      return out_mesh

  triangle_mesh = create_mesh(msh, "triangle", prune_z=True)
  meshio.write("BFS.xdmf", triangle_mesh)

### Preliminary Definitions
Let $\Omega$ be the domain and let $\partial\Omega$ be its boundary, composed by
\begin{equation}
\partial\Omega = \Gamma_i\cup \Gamma_w\cup \Gamma_o
\end{equation}
where $\Gamma_*$ indicates the inlet, wall and outlet boundary.
In this section the mesh, the time discretization parameters and the functional spaces will be defined.

In [ ]:
import numpy as np
from time import process_time, sleep
import time
import tqdm.notebook
# import tqdm.tqdm (uncomment this when executing as a .py)

import ufl
from dolfinx import plot, fem, mesh
from dolfinx.fem import (Function, FunctionSpace, assemble_scalar, form, 
                        locate_dofs_topological, dirichletbc, locate_dofs_geometrical)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import XDMFFile
from dolfinx.mesh import create_rectangle, CellType, locate_entities_boundary
from ufl import dx, grad, inner, div, conj, real, imag, Dx, nabla_grad, dot
from google.colab import files

from mpi4py import MPI
from petsc4py import PETSc


# Parameters
t = 0.0; # time
T = 15. # Final time
dt = 1/50 #time discretization

numsteps = T/dt
saveEvery = 0.2

scale = 1 # meters
xInlet = 0.1 / scale
xOutlet = 2.1 / scale
yBottom = 0.0 / scale
yTop = 0.5 / scale
xTube = 0.4 / scale
yTube = 0.3

Dh = yTube - yBottom

hbar = 0.05; # how to choose it ?

# Mesh Discretization
if gmshFlag == 1:
  with XDMFFile(MPI.COMM_WORLD, "BFS.xdmf", "r") as xdmf:
    domain = xdmf.read_mesh(name="Grid")

tdim = domain.topology.dim;

fdim = tdim - 1

inlet_marker  = 1 
walls_marker  = 2
outlet_marker = 3

boundaries = [(inlet_marker, lambda x: np.isclose(x[0], xInlet)),
              (walls_marker, lambda x: np.logical_and(np.isclose(x[1], yTube), x[0] <= xTube) ),
              (walls_marker, lambda x: np.logical_and(np.isclose(x[0], xTube), x[1] <= yTube) ),
              (walls_marker, lambda x: np.isclose(x[1], yBottom)),
              (walls_marker, lambda x: np.isclose(x[1], yTop)),
              (outlet_marker, lambda x: np.isclose(x[0], xOutlet))]

facet_indices, facet_markers = [], []
for (marker, locator) in boundaries:
    facets = mesh.locate_entities(domain, fdim, locator)
    facet_indices.append(facets)
    facet_markers.append(np.full(len(facets), marker))
facet_indices = np.array(np.hstack(facet_indices), dtype=np.int32)
facet_markers = np.array(np.hstack(facet_markers), dtype=np.int32)
sorted_facets = np.argsort(facet_indices)
facet_tag = mesh.meshtags(domain, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets])

ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tag)

domain.topology.create_connectivity(fdim, tdim)

checkBoundaries = 0

if checkBoundaries == 1:
  with XDMFFile(domain.comm, "meshFacets.xdmf", "w") as checkFacets:
    checkFacets.write_mesh(domain)
    checkFacets.write_meshtags(facet_tag)

  files.download("meshFacets.xdmf")
  files.download("meshFacets.h5")

facets_inlet = np.array(facet_tag.indices[facet_tag.values == inlet_marker])
facets_walls = np.array(facet_tag.indices[facet_tag.values == walls_marker])
facets_outlet = np.array(facet_tag.indices[facet_tag.values == outlet_marker])

# Schrodinger: Functional Space

degSchrodinger = 2

V = FunctionSpace(domain, ("CG", degSchrodinger))

psi = ufl.TrialFunction(V)
v   = ufl.TestFunction(V)

Psi1old = Function(V)
Psi2old = Function(V)

Psi1Tilde = Function(V)
Psi2Tilde = Function(V)

Psi1TildeNormalized = Function(V)
Psi2TildeNormalized = Function(V)

Psi1 = Function(V)
Psi1.name = "Psi1"

Psi2 = Function(V)
Psi2.name = "Psi2"

# Schrodinger: defining the inlet boundary condition
c1 = 1.0 + 0.0 * 1j
c2 = 0.01 + 0.0 * 1j

constantNorm = np.conj(c1) * c1 + np.conj(c2) * c2
c1 /= constantNorm
c2 /= constantNorm

# Schrodinger: initialization

Psi1old.interpolate(lambda x: c1 + x[0] * 0.0 + x[1] * 0.0)
Psi2old.interpolate(lambda x: c2 + x[0] * 0.0 + x[1] * 0.0) 

Psi1.interpolate(lambda x: c1 + x[0] * 0.0 + x[1] * 0.0)
Psi2.interpolate(lambda x: c2 + x[0] * 0.0 + x[1] * 0.0) 

# Schrodinger pressure functional space
QSchrodinger = FunctionSpace(domain, ("CG", degSchrodinger - 1))

pSchrodinger = ufl.TrialFunction(QSchrodinger)
phiSchrodinger = ufl.TestFunction(QSchrodinger)

pressureSchrodinger = Function(QSchrodinger)
pressure = Function(QSchrodinger)
pressure.name = "p"

dofs_inlet_QSchrodinger = locate_dofs_topological(QSchrodinger, fdim, facets_inlet)
dofs_outlet_QSchrodinger = locate_dofs_topological(QSchrodinger, fdim, facets_outlet)

pressureBC_Schrodinger = Function(QSchrodinger)

pressureBC_Schrodinger.interpolate(lambda x: 0.0 + 0.0 * 1j + 0.0 * x[0] + 0.0 * x[1])

bc_PressureSchrodinger = [dirichletbc(pressureBC_Schrodinger, dofs_inlet_QSchrodinger)]

# Velocity Vector Functional Space
gdim = domain.geometry.dim
deg = 2

vectorElement = ufl.VectorElement("CG", domain.ufl_cell(), deg, gdim)
U_vectorSpace = FunctionSpace(domain, vectorElement) 

uNew = Function(U_vectorSpace)
uNew.name = "U"

uTilde = Function(U_vectorSpace)

u = ufl.TrialFunction(U_vectorSpace)
eta = ufl.TestFunction(U_vectorSpace)